In [1]:
import requests
import pandas as pd

url = "https://www.land.mlit.go.jp/webland/api/TradeListSearch"
parameters = {
    "from": "20111",
    "to": "20234",
    "area": "14",
    "city":"14101",
}

response = requests.get(url, params=parameters)

if response.status_code == 200:
    data = response.json()  # JSON形式のデータを取得
    df = pd.DataFrame(data['data'])
else:
    print("Request was not successful. Status code:", response.status_code)


In [3]:
df.Period.unique()

array(['2021年第４四半期', '2020年第４四半期', '2023年第２四半期', '2022年第２四半期',
       '2021年第２四半期', '2020年第３四半期', '2020年第２四半期', '2019年第３四半期',
       '2019年第２四半期', '2018年第４四半期', '2018年第３四半期', '2018年第２四半期',
       '2018年第１四半期', '2022年第４四半期', '2019年第１四半期', '2022年第３四半期',
       '2022年第１四半期', '2023年第１四半期', '2021年第３四半期', '2021年第１四半期',
       '2020年第１四半期', '2019年第４四半期', '2014年第３四半期', '2014年第２四半期',
       '2014年第４四半期', '2014年第１四半期', '2017年第４四半期', '2017年第３四半期',
       '2017年第２四半期', '2017年第１四半期', '2011年第１四半期', '2011年第４四半期',
       '2011年第３四半期', '2011年第２四半期', '2015年第１四半期', '2015年第２四半期',
       '2015年第４四半期', '2015年第３四半期', '2016年第４四半期', '2016年第３四半期',
       '2016年第２四半期', '2016年第１四半期', '2012年第１四半期', '2012年第４四半期',
       '2012年第３四半期', '2012年第２四半期', '2013年第１四半期', '2013年第２四半期',
       '2013年第３四半期', '2013年第４四半期'], dtype=object)

In [4]:
def RemoveM2(df_, needRemoveVal:dict):
    # target_wordの文字を含むデータのtarget_wordを除去
    # 新規列を追加し、除去した行はフラグを立てる
    for col, target_word in needRemoveVal.items():
        new_col = col+'_over_flag'
        target_row = df_.loc[:, col].str.contains(target_word).fillna(False)
        # print(col, target_word, sum(target_row))
        df_[new_col] = 0
        df_.loc[target_row, new_col] = 1
        df_[col] = df_.loc[:,col].str.replace(target_word,'')
        
    return df_

def changeDataType(df_,changeDataTypeVal:dict):
    for col,new_data_type in changeDataTypeVal.items():
        # print(col,new_data_type)
        df_[col] = df_.loc[:,col].astype(new_data_type)
    return df_

def ExtractRow(df_, extractCondionDic):
    for col, targetValue in extractCondionDic.items():
        if targetValue =='Nan': # Nanと他のも選びたい場合汎用性ないけど、とりあえず
            df_ = df_[df_[col].isna()]
        else:
            target_row = df_[col].isin(targetValue)
            df_ = df_[target_row]
    return df_


NameError: name 'df2' is not defined

In [201]:
drop_cols = ['PricePerUnit', 'Purpose', 'Direction', 'Classification']

# val: m2が含まれる値を、何に置換するか
needRemoveVal = {
    'Frontage': 'm以上', 
    'Area': '㎡以上',
    'TotalFloorArea':'㎡以上'
}
changeDataTypeVal = {
    'TradePrice':'int',
    'Frontage':'float',
    'Area':'int',
    'UnitPrice':'float',
    'TotalFloorArea':'float',
    'Breadth':'float',
    'CoverageRatio':'float',
    'FloorAreaRatio':'float'
}
extractCondionDic = {
    'Type':['宅地(土地と建物)', '中古マンション等', '宅地(土地)'],
    'Remarks': 'Nan'
    }

df2 = df.drop(drop_cols, axis=1)
df2 = RemoveM2(df2, needRemoveVal)
df2 = changeDataType(df2, changeDataTypeVal)
print(df2.shape)
df2 = ExtractRow(df2, extractCondionDic)
df2.shape
# df2.dtypes

(19877, 26)


(19037, 26)

In [183]:
df2.head()

,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,Area,LandShape,Frontage,...,FloorAreaRatio,Period,Use,FloorPlan,Renovation,Remarks,UnitPrice,Frontage_over_flag,Area_over_flag,TotalFloorArea_over_flag
0,宅地(土地と建物),住宅地,14101,神奈川県,横浜市鶴見区,朝日町,40000000,60,長方形,6.0,...,200.0,2022年第２四半期,NaN,NaN,NaN,NaN,NaN,0,0,0
1,宅地(土地と建物),住宅地,14101,神奈川県,横浜市鶴見区,朝日町,33000000,50,長方形,4.5,...,200.0,2022年第４四半期,住宅,NaN,NaN,NaN,NaN,0,0,0
2,宅地(土地と建物),住宅地,14101,神奈川県,横浜市鶴見区,朝日町,2200000,35,正方形,6.0,...,200.0,2022年第４四半期,住宅,NaN,NaN,NaN,NaN,0,0,0
3,宅地(土地と建物),住宅地,14101,神奈川県,横浜市鶴見区,朝日町,11000000,75,長方形,9.0,...,200.0,2022年第４四半期,NaN,NaN,NaN,NaN,NaN,0,0,0
4,中古マンション等,NaN,14101,神奈川県,横浜市鶴見区,朝日町,34000000,60,NaN,NaN,...,200.0,2022年第３四半期,住宅,３ＬＤＫ,改装済,NaN,NaN,0,0,0


In [76]:
target_col = 'TotalFloorArea'
i = 0
try:
    df.loc[:,target_col].astype('int')
    print('intOK')
    i = 1
except:
    print('int不可')
    
if i == 0:
    try:
        df.loc[:,target_col].astype('float')
        print('floatOK')
    except:
        print('float不可')
df.loc[:,target_col].unique()

int不可
float不可


array(['300', nan, '1700', '1800', '470', '990', '125', '230', '95',
       '600', '2000㎡以上', '240', '580', '120', '190', '370', '260', '130',
       '550', '185', '195', '90', '140', '80', '170', '290', '250', '155',
       '105', '730', '610', '280', '145', '210', '1100', '980', '115',
       '560', '320', '1000', '180', '830', '65', '680', '640', '840',
       '490', '420', '380', '740', '430', '220', '390', '1500', '45',
       '135', '630', '450', '920', '110', '1300', '710', '500', '85',
       '650', '340', '1200', '900', '530', '160', '175', '200', '810',
       '150', '100', '690', '1900', '660', '780', '930', '870', '410',
       '165', '440', '890', '330', '700', '520', '570', '720', '460',
       '350', '1400', '75', '70', '55', '360', '310', '880', '820', '670',
       '480', '400', '910', '590', '760', '750', '950', '60', '960',
       '790', '270', '850', '510', '540', '620', '25', '770', '20', '860',
       '35', '1600', '30', '40', '800', '50', '10', '940'], dtype=obje

In [184]:
df2.pivot_table(index='Remarks', values='Type',aggfunc='count').sort_values('Type', ascending=False)

,Type
Remarks,
私道を含む取引,492
調停・競売等,132
隣地の購入,24
関係者間取引,22
その他事情有り,15
調停・競売等、私道を含む取引,2
隣地の購入、調停・競売等,2
関係者間取引、私道を含む取引,1
隣地の購入、私道を含む取引,1
